# Structured Streaming using Python DataFrames API

Apache Spark 2.0 adds the first version of a new higher-level stream processing API, Structured Streaming. In this notebook we are going to take a quick look at how to use DataFrame API to build Structured Streaming applications. We want to compute real-time metrics like running counts and windowed counts on a stream of timestamped actions.

To run this notebook, import it to Databricks Community Edition and attach it to a **Spark 2.1 (Scala 2.10)** cluster.

## Data
We will be using the same flights datasets as the previous sections.
The first few cells are simply the same data loading and cleaning you have already seen.

In [0]:
# To make sure that this notebook is being run on a Spark 2.0 cluster, let's see if we can access the SparkSession - the new entry point of Apache Spark 2.0.
# If this fails, then you are not connected to a Spark 2.0 cluster. Please recreate your cluster and select the version to be "Spark 2.0 (Scala 2.10)".
spark

Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# Set File Paths
tripdelaysFilePath = "/databricks-datasets/flights/departuredelays.csv"
airportsnaFilePath = "/databricks-datasets/flights/airport-codes-na.txt"

# Obtain airports dataset
airportsna = sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true', delimiter='\t').load(airportsnaFilePath)
airportsna.registerTempTable("airports_na")
airportsna.show()

# Obtain departure Delays data
departureDelay = sqlContext.read.format("com.databricks.spark.csv").options(header='true').load(tripdelaysFilePath)
departureDelay.show()
departureDelays = sqlContext.read.format("com.databricks.spark.csv").schema(departureDelay.schema).options(header='true').load(tripdelaysFilePath)
departureDelays.show()
departureDelays.registerTempTable("departureDelays")
departureDelays.cache()


+-----------+-----+-------+----+
 City|State|Country|IATA|
+-----------+-----+-------+----+
 Abbotsford| BC| Canada| YXX|
 Aberdeen| SD| USA| ABR|
 Abilene| TX| USA| ABI|
 Akron| OH| USA| CAK|
 Alamosa| CO| USA| ALS|
 Albany| GA| USA| ABY|
 Albany| NY| USA| ALB|
Albuquerque| NM| USA| ABQ|
 Alexandria| LA| USA| AEX|
 Allentown| PA| USA| ABE|
 Alliance| NE| USA| AIA|
 Alpena| MI| USA| APN|
 Altoona| PA| USA| AOO|
 Amarillo| TX| USA| AMA|
Anahim Lake| BC| Canada| YAA|
 Anchorage| AK| USA| ANC|
 Appleton| WI| USA| ATW|
 Arviat| NWT| Canada| YEK|
 Asheville| NC| USA| AVL|
 Aspen| CO| USA| ASE|
+-----------+-----+-------+----+
only showing top 20 rows

+--------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
01011245| 6| 602| ABE| ATL|
01020600| -8| 369| ABE| DTW|
01021245| -2| 602| ABE| ATL|
01020605| -4| 602| ABE| ATL|
01031245| -4| 602| ABE| ATL|
01030605| 0| 602| ABE| ATL|
01041243| 10| 602| ABE| ATL|
01040605| 28| 602| ABE| ATL|
01051245| 88| 602| ABE| ATL|
01050605| 9| 602| ABE| ATL|
01061215| -6| 602| ABE| ATL|
01061725| 69| 602| ABE| ATL|
01061230| 0| 369| ABE| DTW|
01060625| -3| 602| ABE| ATL|
01070600| 0| 369| ABE| DTW|
01071725| 0| 602| ABE| ATL|
01071230| 0| 369| ABE| DTW|
01070625| 0| 602| ABE| ATL|
01071219| 0| 569| ABE| ORD|
01080600| 0| 369| ABE| DTW|
+--------+-----+--------+------+-----------+
only showing top 20 rows

+--------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
01011245| 6| 602| ABE| ATL|
01020600| -8| 369| ABE| DTW|
01021245| -2| 602| ABE| ATL|
01020605| -4| 602| ABE| ATL|
01031245| -4| 602| ABE| ATL|
01030605| 0| 602| ABE| ATL|
01041243| 10| 602| ABE| ATL|
01040605| 28| 602| ABE| ATL|
01051245| 88| 602| ABE| ATL|
01050605| 9| 602| ABE| ATL|
01061215| -6| 602| ABE| ATL|
01061725| 69| 602| ABE| ATL|
01061230| 0| 369| ABE| DTW|
01060625| -3| 602| ABE| ATL|
01070600| 0| 369| ABE| DTW|
01071725| 0| 602| ABE| ATL|
01071230| 0| 369| ABE| DTW|
01070625| 0| 602| ABE| ATL|
01071219| 0| 569| ABE| ORD|
01080600| 0| 369| ABE| DTW|
+--------+-----+--------+------+-----------+
only showing top 20 rows

Out[3]: DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]

In [0]:


# Available IATA codes from the departuredelays sample dataset
tripIATA = sqlContext.sql("select distinct iata from (select distinct origin as iata from departureDelays union all select distinct destination as iata from departureDelays) a")
tripIATA.registerTempTable("tripIATA")
tripIATA.show()
# Only include airports with atleast one trip from the departureDelays dataset
airports = sqlContext.sql("select f.IATA, f.City, f.State, f.Country from airports_na f join tripIATA t on t.IATA = f.IATA")
airports.registerTempTable("airports")
airports.cache()
airports.show()

+----+
iata|
+----+
 GEG|
 BUR|
 GRB|
 GTF|
 GRR|
 EUG|
 GSO|
 BTM|
 COD|
 FAR|
 FSM|
 DCA|
 CID|
 GTR|
 HLN|
 EVV|
 CRW|
 CDV|
 CMH|
 CAK|
+----+
only showing top 20 rows

+----+-------------------+-----+-------+
IATA| City|State|Country|
+----+-------------------+-----+-------+
 INL|International Falls| MN| USA|
 MSY| New Orleans| LA| USA|
 GEG| Spokane| WA| USA|
 BUR| Burbank| CA| USA|
 SNA| Orange County| CA| USA|
 GRB| Green Bay| WI| USA|
 GTF| Great Falls| MT| USA|
 IDA| Idaho Falls| ID| USA|
 GRR| Grand Rapids| MI| USA|
 JLN| Joplin| MO| USA|
 EUG| Eugene| OR| USA|
 GSO| Greensboro| NC| USA|
 MYR| Myrtle Beach| SC| USA|
 PVD| Providence| RI| USA|
 OAK| Oakland| CA| USA|
 BTM| Butte| MT| USA|
 COD| Cody| WY| USA|
 FAR| Fargo| ND| USA|
 FSM| Fort Smith| AR| USA|
 MQT| Marquette| MI| USA|
+----+-------------------+-----+-------+
only showing top 20 rows

In [0]:
# Build `departureDelays_geo` DataFrame
#  Obtain key attributes such as Date of flight, delays, distance, and airport information (Origin, Destination)  
departureDelays_geo = sqlContext.sql("select cast(f.date as int) as tripid, cast(concat(concat(concat(concat(concat(concat('2014-', concat(concat(substr(cast(f.date as string), 1, 2), '-')), substr(cast(f.date as string), 3, 2)), ' '), substr(cast(f.date as string), 5, 2)), ':'), substr(cast(f.date as string), 7, 2)), ':00') as timestamp) as `localdate`, cast(f.delay as int), cast(f.distance as int), f.origin as src, f.destination as dst, o.city as city_src, d.city as city_dst, o.state as state_src, d.state as state_dst from departuredelays f join airports o on o.iata = f.origin join airports d on d.iata = f.destination") 

# RegisterTempTable
#departureDelays_geo.repartition(50).write.json("departureDelays_json")
departureDelays_geo.show()
# Cache and Count
departureDelays_geo.cache()
departureDelays_geo.count()



+-------+-------------------+-----+--------+---+---+---------+--------+---------+---------+
 tripid| localdate|delay|distance|src|dst| city_src|city_dst|state_src|state_dst|
+-------+-------------------+-----+--------+---+---+---------+--------+---------+---------+
1011245|2014-01-01 12:45:00| 6| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1020600|2014-01-02 06:00:00| -8| 369|ABE|DTW|Allentown| Detroit| PA| MI|
1021245|2014-01-02 12:45:00| -2| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1020605|2014-01-02 06:05:00| -4| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1031245|2014-01-03 12:45:00| -4| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1030605|2014-01-03 06:05:00| 0| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1041243|2014-01-04 12:43:00| 10| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1040605|2014-01-04 06:05:00| 28| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1051245|2014-01-05 12:45:00| 88| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1050605|2014-01-05 06:05:00| 9| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1061215|2014-01-06 12:15:00| -6| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1061725|2014-01-06 17:25:00| 69| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1061230|2014-01-06 12:30:00| 0| 369|ABE|DTW|Allentown| Detroit| PA| MI|
1060625|2014-01-06 06:25:00| -3| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1070600|2014-01-07 06:00:00| 0| 369|ABE|DTW|Allentown| Detroit| PA| MI|
1071725|2014-01-07 17:25:00| 0| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1071230|2014-01-07 12:30:00| 0| 369|ABE|DTW|Allentown| Detroit| PA| MI|
1070625|2014-01-07 06:25:00| 0| 602|ABE|ATL|Allentown| Atlanta| PA| GA|
1071219|2014-01-07 12:19:00| 0| 569|ABE|ORD|Allentown| Chicago| PA| IL|
1080600|2014-01-08 06:00:00| 0| 369|ABE|DTW|Allentown| Detroit| PA| MI|
+-------+-------------------+-----+--------+---+---+---------+--------+---------+---------+
only showing top 20 rows

Out[6]: 1361141

## Sample Data
We are now going to save the contents of the departureDelays_geo dataframe to JSON files, and use it to simulate streaming data.

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "200")  # Use 200 partitions for shuffling
#departureDelays_geo.orderBy("localdate").write.json("departureDelays_json")
departureDelays_geo_schema = departureDelays_geo.schema
departureDelays_geo_schema

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3819035396542450> in <module> 
 1 spark . conf . set ( "spark.sql.shuffle.partitions" , "200" ) # Use 200 partitions for shuffling 
 ----> 2 departureDelays_geo . orderBy ( "localdate" ) . write . json ( "departureDelays_json" ) 
 3 departureDelays_geo_schema = departureDelays_geo . schema
 4 departureDelays_geo_schema

 /databricks/spark/python/pyspark/sql/readwriter.py in json (self, path, mode, compression, dateFormat, timestampFormat, lineSep, encoding, ignoreNullFields) 
 1238 compression = compression , dateFormat = dateFormat , timestampFormat = timestampFormat , 
 1239 lineSep=lineSep, encoding=encoding, ignoreNullFields=ignoreNullFields)
 -> 1240 self . _jwrite . json ( path ) 
 1241 
 1242 def parquet ( self , path , mode = None , partitionBy = None , compression = None ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 114 # Hide where the exception came from that shows a non-Pythonic 
 115 # JVM exception message. 
 --> 116 raise converted from None 
 117 else : 
 118 raise 

 AnalysisException : path dbfs:/departureDelays_json already exists.

In [0]:
%fs head /departureDelays_json/part-00015-tid-3892456853287454747-116e3f75-1d66-48b6-a3fc-2ac7fdf6cc59-8512-1-c000.json


[Truncated to first 65536 bytes]
{"tripid":3230914,"localdate":"2014-03-23T09:14:00.000Z","delay":-6,"distance":704,"src":"TPA","dst":"IAD","city_src":"Tampa","city_dst":"Washington DC","state_src":"FL"}
{"tripid":3230914,"localdate":"2014-03-23T09:14:00.000Z","delay":-3,"distance":288,"src":"MEM","dst":"ATL","city_src":"Memphis","city_dst":"Atlanta","state_src":"TN","state_dst":"GA"}
{"tripid":3230914,"localdate":"2014-03-23T09:14:00.000Z","delay":-4,"distance":788,"src":"MSP","dst":"ATL","city_src":"Minneapolis","city_dst":"Atlanta","state_src":"MN","state_dst":"GA"}
{"tripid":3230914,"localdate":"2014-03-23T09:14:00.000Z","delay":-10,"distance":121,"src":"MLI","dst":"ORD","city_src":"Moline","city_dst":"Chicago","state_src":"IL","state_dst":"IL"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":-3,"distance":351,"src":"SMF","dst":"SNA","city_src":"Sacramento","city_dst":"Orange County","state_src":"CA","state_dst":"CA"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":-7,"distance":151,"src":"ORD","dst":"GRB","city_src":"Chicago","city_dst":"Green Bay","state_src":"IL","state_dst":"WI"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":-8,"distance":71,"src":"KTN","dst":"WRG","city_src":"Ketchikan","city_dst":"Wrangell","state_src":"AK","state_dst":"AK"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":-6,"distance":257,"src":"ORD","dst":"CMH","city_src":"Chicago","city_dst":"Columbus","state_src":"IL","state_dst":"OH"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":-3,"distance":2272,"src":"LAX","dst":"LIH","city_src":"Los Angeles","city_dst":"Lihue, Kauai","state_src":"CA","state_dst":"HI"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":0,"distance":373,"src":"MAF","dst":"IAH","city_src":"Midland","city_dst":"Houston","state_src":"TX","state_dst":"TX"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":519,"distance":343,"src":"OKC","dst":"IAH","city_src":"Oklahoma City","city_dst":"Houston","state_src":"OK","state_dst":"TX"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":-9,"distance":4312,"src":"EWR","dst":"HNL","city_src":"Newark","city_dst":"Honolulu, Oahu","state_src":"NJ","state_dst":"HI"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":3,"distance":336,"src":"LAS","dst":"SJC","city_src":"Las Vegas","city_dst":"San Jose","state_src":"NV","state_dst":"CA"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":50,"distance":411,"src":"ORD","dst":"BUF","city_src":"Chicago","city_dst":"Buffalo","state_src":"IL","state_dst":"NY"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":0,"distance":960,"src":"FLL","dst":"AUS","city_src":"Fort Lauderdale","city_dst":"Austin","state_src":"FL","state_dst":"TX"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":-1,"distance":121,"src":"ORD","dst":"MLI","city_src":"Chicago","city_dst":"Moline","state_src":"IL","state_dst":"IL"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":-4,"distance":307,"src":"SFO","dst":"LGB","city_src":"San Francisco","city_dst":"Long Beach","state_src":"CA","state_dst":"CA"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":-1,"distance":844,"src":"MCO","dst":"ISP","city_src":"Orlando","city_dst":"Islip","state_src":"FL","state_dst":"NY"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":-7,"distance":243,"src":"PHL","dst":"BOS","city_src":"Philadelphia","city_dst":"Boston","state_src":"PA","state_dst":"MA"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":-3,"distance":917,"src":"DFW","dst":"LAS","city_src":"Dallas","city_dst":"Las Vegas","state_src":"TX","state_dst":"NV"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00.000Z","delay":-5,"distance":205,"src":"LAX","dst":"LAS","city_src":"Los Angeles","city_dst":"Las Vegas","state_src":"CA","state_dst":"NV"}
{"tripid":3230915,"localdate":"2014-03-23T09:15:00

There are 200 JSON files in the directory. Let's see what each JSON file contains.

In [0]:
%fs ls /departureDelays_json/part-00015-tid-3892456853287454747-116e3f75-1d66-48b6-a3fc-2ac7fdf6cc59-8512-1-c000.json

path,name,size
dbfs:/departureDelays_json/part-00015-tid-3892456853287454747-116e3f75-1d66-48b6-a3fc-2ac7fdf6cc59-8512-1-c000.json,part-00015-tid-3892456853287454747-116e3f75-1d66-48b6-a3fc-2ac7fdf6cc59-8512-1-c000.json,14932631


## Batch/Interactive Processing
The usual first step in attempting to process the data is to interactively query the data. Let's define a static DataFrame on the files, and give it a table name.

In [0]:
from pyspark.sql.types import *

inputPath = "/departureDelays_json"
#    .schema(departureDelays_geo_schema)


# Static DataFrame representing data in the JSON files
staticInputDF = (
  spark
    .read
    .json(inputPath)
)

display(staticInputDF)

city_dst,city_src,delay,distance,dst,localdate,src,state_dst,state_src,tripid
Denver,Chicago,6,772,DEN,2014-02-19T20:56:00.000Z,ORD,CO,IL,2192056
Orange County,San Francisco,-4,323,SNA,2014-02-19T20:57:00.000Z,SFO,CA,CA,2192057
Washington DC,Chicago,37,532,DCA,2014-02-19T20:57:00.000Z,ORD,null,IL,2192057
New York,Fort Lauderdale,84,935,LGA,2014-02-19T20:57:00.000Z,FLL,NY,FL,2192057
Seattle,San Francisco,-7,590,SEA,2014-02-19T20:57:00.000Z,SFO,WA,CA,2192057
Flagstaff,Phoenix,-4,104,FLG,2014-02-19T20:57:00.000Z,PHX,AZ,AZ,2192057
Houston,Cleveland,-6,948,IAH,2014-02-19T20:58:00.000Z,CLE,TX,OH,2192058
Richmond,Chicago,98,558,RIC,2014-02-19T20:58:00.000Z,ORD,VA,IL,2192058
Peoria,Denver,38,689,PIA,2014-02-19T20:58:00.000Z,DEN,IL,CO,2192058
Orange County,Denver,5,735,SNA,2014-02-19T20:59:00.000Z,DEN,CA,CO,2192059


Now we can compute the number of flights leaving from each state, with 5 hour time windows. To do this, we will group by the `state_src` column and 5 hour windows over the `localdate` column.

In [0]:
from pyspark.sql.functions import *      # for window() function

staticCountsDF = (
  staticInputDF
    .groupBy(
      staticInputDF.state_src, 
      window(staticInputDF.localdate, "5 hours"))    
    .count()
)

staticCountsDF.printSchema()
staticCountsDF.sort(desc("count")).show()
staticCountsDF.cache()

# Register the DatFrame as table 'static_counts'
staticCountsDF.createOrReplaceTempView("static_counts")

root
-- state_src: string (nullable = true)
-- window: struct (nullable = false)
 |-- start: timestamp (nullable = true)
 |-- end: timestamp (nullable = true)
-- count: long (nullable = false)

+---------+--------------------+-----+
state_src| window|count|
+---------+--------------------+-----+
 CA|{2014-01-06 06:00...| 760|
 CA|{2014-02-20 06:00...| 755|
 CA|{2014-02-10 06:00...| 752|
 CA|{2014-01-16 06:00...| 750|
 CA|{2014-01-31 06:00...| 749|
 CA|{2014-03-07 06:00...| 749|
 CA|{2014-03-27 06:00...| 745|
 CA|{2014-03-17 06:00...| 744|
 CA|{2014-03-12 06:00...| 733|
 CA|{2014-02-25 06:00...| 729|
 CA|{2014-02-05 06:00...| 723|
 CA|{2014-01-21 06:00...| 719|
 CA|{2014-01-02 07:00...| 706|
 CA|{2014-03-24 08:00...| 703|
 CA|{2014-03-14 08:00...| 701|
 CA|{2014-02-21 07:00...| 700|
 CA|{2014-03-03 07:00...| 697|
 CA|{2014-01-03 08:00...| 695|
 TX|{2014-03-24 08:00...| 695|
 CA|{2014-02-27 08:00...| 694|
+---------+--------------------+-----+
only showing top 20 rows

Now we can directly use SQL to query the table. For example, here is a timeline of the windowed counts of flights leaving from each state.

In [0]:
%sql select state_src, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts order by count desc, time, state_src

state_src,time,count
CA,Jan-06 11:00,760
CA,Feb-20 11:00,755
CA,Feb-10 11:00,752
CA,Jan-16 11:00,750
CA,Jan-31 11:00,749
CA,Mar-07 11:00,749
CA,Mar-27 11:00,745
CA,Mar-17 11:00,744
CA,Mar-12 11:00,733
CA,Feb-25 11:00,729


## Stream Processing 
Now that we have analyzed the data interactively, let's convert this to a streaming query that continuously updates as data comes. Since we just have a static set of files, we are going to emulate a stream from them by reading one file at a time, in the chronological order they were created. The query we have to write is pretty much the same as the interactive query above.

In [0]:
from pyspark.sql.functions import *

inputPath = "/departureDelays_json"
departureDelays_geo_schema = departureDelays_geo.schema
# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(departureDelays_geo_schema) # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.state_src, 
      window(streamingInputDF.localdate, "5 hours"))    
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

Out[29]: True

As you can see, `streamingCountsDF` is a streaming Dataframe (`streamingCountsDF.isStreaming` was `true`). You can start streaming computation, by defining the sink and starting it. 
In our case, we want to interactively query the counts (same queries as above), so we will set the complete set of 1 hour counts to be in a in-memory table (note that this for testing purpose only in Spark 2.0).

In [0]:
from pyspark.sql.streaming import *

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "1")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3819035396542463> in <module> 
 4 streamingCountsDF
 5 . writeStream
 ----> 6 . trigger ( trigger . ProcessingTime ( "3 secondes" ) ) 
 7 . format ( "memory" ) # memory = store in-memory table (for testing only in Spark 2.0) 
 8 . queryName ( "counts" ) # counts = name of the in-memory table 

 NameError : name 'trigger' is not defined

`query` is a handle to the streaming query that is running in the background. This query is continuously picking up files and updating the windowed counts. 

Note the status of query in the above cell. Both the `Status: ACTIVE` and the progress bar shows that the query is active. 
Furthermore, if you expand the `>Details` above, you will find the number of files they have already processed. 

Let's wait a bit for a few files to be processed and then interactively query the in-memory `counts` table.

In [0]:
from time import sleep
sleep(5)  # wait a bit for computation to start

In [0]:
%sql select state_src, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by count desc, time, state_src

state_src,time,count
CA,Jan-06 11:00,760
CA,Feb-20 11:00,755
CA,Feb-10 11:00,752
CA,Jan-16 11:00,750
CA,Jan-31 11:00,749
CA,Mar-07 11:00,749
CA,Mar-27 11:00,745
CA,Mar-17 11:00,744
CA,Mar-12 11:00,733
CA,Feb-25 11:00,729


We see the timeline of windowed counts (similar to the static one ealrier) building up. If we keep running this interactive query repeatedly, we will see the latest updated counts which the streaming query is updating in the background.

In [0]:
sleep(5)  # wait a bit more for more data to be computed

In [0]:
%sql select state_src, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, state_src

If you keep running the above query repeatedly, you will observe that earlier dates don't appear after later dates have been observed, as expected in a data stream where data appears in time-sorted order. This shows that Structured Streaming ensures **prefix integrity**. Structured streaming also has settings to control how out-of-order data is handled. Read the blog posts linked below if you want to know more.

Note that there are only a few files, so after consuming all of them there will be no updates to the counts. Rerun the query if you want to interact with the streaming query again.

Finally, you can stop the query running in the background, either by clicking on the 'Cancel' link in the cell of the query, or by executing `query.stop()`. Either way, when the query is stopped, the status of the corresponding cell above will automatically update to `TERMINATED`.